<a href="https://colab.research.google.com/github/ras1364/DetectFakeReviewsUsing-Sentence_BERT/blob/main/TrainAndTestSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is created to separate training data and test data. In this code, training and test data are separated and stored in separate files.

In [1]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from string import ascii_lowercase
from collections import Counter
import itertools, nltk, snowballstemmer, re

In [2]:
data = pd.read_csv("/content/sample_data/deceptive-opinion.csv")
#data.head()

In [3]:
data['polarity'] = np.where(data['polarity']=='positive',1,0)
data['deceptive'] = np.where(data['deceptive']=='truthful', 1,0)
#data.head()

In [4]:
df = data.sample(frac=1)
#df.head()

In [5]:
#df.describe()

In [6]:
def create_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return [1,1]
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return [1,0]
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return [0,1]
    else:
        return [0,0]
    
def specific_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return "TRUE_POSITIVE"
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return "FALSE_POSITIVE"
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return "TRUE_NEGATIVE"
    else:
        return "FALSE_NEGATIVE"

data['final_class'] = data.apply(create_class, axis=1)
data['given_class'] = data.apply(specific_class, axis=1)
#data['final_class']

In [7]:
# Number of sentece in each review 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

for sen in data.text:
  data["NumberOfSentece"] = len(sent_tokenize(sen))
#data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
sum =0
for a in data.NumberOfSentece:
  sum += a
print("Average number of reviews : ", sum/len(data), "  Sum = ",sum , "  Length = ", len(data) )

Average number of reviews :  13.0   Sum =  20800   Length =  1600


In [9]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
data['given_class']= label_encoder.fit_transform(data['given_class']) 
data['given_class'].unique()

array([3, 1, 2, 0])

In [10]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
Y = data['given_class']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
data.head()

,deceptive,hotel,polarity,source,text,final_class,given_class,NumberOfSentece
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",3,13
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",3,13
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",3,13
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",3,13
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",3,13


In [12]:
from sklearn.model_selection import train_test_split
X = data["text"]
y = data["deceptive"]


X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y)

In [13]:
X_train.to_csv('/content/sample_data/X_train.csv')
y_train.to_csv('/content/sample_data/y_train.csv')
X_test.to_csv('/content/sample_data/X_test.csv')
y_test.to_csv('/content/sample_data/y_test.csv')